In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as ET
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
def parse_relative_time(relative_time):

    if relative_time is None:
        return None

    relative_time = relative_time.lower()
    now = datetime.now()

    if "segundos" in relative_time or "minutos" in relative_time:
        # Extrair o número de minutos
        minutes = int(relative_time.split()[1])
        return now - timedelta(minutes=minutes)

    elif "horas" in relative_time:
        # Extrair o número de horas
        hours = int(relative_time.split()[1])
        return now - timedelta(hours=hours)

    elif "ontem" in relative_time:
        # Ontem
        return now - timedelta(days=1)

    elif "dia" in relative_time or "dias" in relative_time:
        # Extrair o número de dias
        days = int(relative_time.split()[1])
        return now - timedelta(days=days)

    elif "semana" in relative_time or "semanas" in relative_time:
        # Extrair o número de semanas
        weeks = int(relative_time.split()[1])
        return now - timedelta(weeks=weeks)

    elif "mês" in relative_time or "meses" in relative_time:
        # Extrair o número de meses
        months = int(relative_time.split()[1])
        # Assume-se que um "mês" tem 30 dias para simplificar
        return now - timedelta(days=months * 30)

    elif "/" in relative_time:
        # Converter a string de data para um objeto datetime
        data = datetime.strptime(relative_time, '%d/%m/%Y %Hh%M')
        # Formatar a data no formato 'ano-mês-dia hora:minuto'
        data_formatada = data.strftime('%Y-%m-%d %H:%M')
        return data_formatada

    else:
        return None


### Extrair Notícias Brazil Journal

In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Função para obter o primeiro parágrafo e a data de uma URL
def obter_dados_noticia(url):
    # Fazendo a solicitação GET para a URL
    response = requests.get(url)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Obtendo o conteúdo HTML
        html = response.text

        # Parseando o HTML com BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Encontrando o primeiro parágrafo dentro da classe 'post-content-text'
        primeiro_paragrafo = soup.find("div", class_="post-content-text").find("p")

        # Encontrando a data dentro da classe 'post-header-info'
        data_element = soup.find("div", class_="post-header-info").find("time")

        # Verificando se o primeiro parágrafo e a data foram encontrados
        if primeiro_paragrafo and data_element:
            # Obtendo o texto do primeiro parágrafo e da data
            descricao = primeiro_paragrafo.text.strip()
            data = data_element.text.strip()
            return descricao, data
        else:
            return "Descrição não disponível", "Data não disponível"
    else:
        return "Falha ao acessar a página. Status code:" + str(response.status_code), "Data não disponível"

# Função para extrair dados das notícias do Brazil Journal
def extrair_dados_brazil_journal():
    # URL para fazer a solicitação POST
    url = "https://braziljournal.com/wp-admin/admin-ajax.php"

    # Payload da solicitação POST
    payload = {
        's': 'vulca',
        'action': 'searchwp_live_search',
        'swpengine': 'default',
        'swpquery': 'vulca',
        'origin_id': '322'
    }

    # Fazendo a solicitação POST
    response = requests.post(url, data=payload)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Obtendo o conteúdo HTML
        html = response.text

        # Parseando o HTML com BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Encontrando todas as notícias
        articles = soup.find_all("article", class_="boxarticle")

        # Lista para armazenar os dados de todas as notícias
        all_data = []

        # Iterando sobre cada notícia para extrair título, link e imagem
        for article in articles:
            title_elem = article.find("h2", class_="boxarticle-infos-title")
            link_elem = article.find("a", class_="boxarticle-img")['href']
            img_elem = article.find("img")['src']

            # Definindo valores padrão para elementos ausentes
            title = title_elem.text.strip() if title_elem else "Título não disponível"
            link = link_elem if link_elem else "Link não disponível"
            img = img_elem if img_elem else "Imagem não disponível"

            # Obtendo o primeiro parágrafo e a data da descrição da notícia
            descricao, data = obter_dados_noticia(link)

            # Adicionando os dados da notícia à lista de dados de todas as notícias
            all_data.append({
                "titulo": title,
                "url": link,
                "img": img,
                "descricao": descricao,
                "data": data,
                "fonte": "Brazil Journal"
            })

        # Criando o DataFrame com os dados de todas as notícias
        df_brazil_journal = pd.DataFrame(all_data)

        # Exibindo o DataFrame
        print(df_brazil_journal)
    else:
        print("Falha ao fazer a solicitação. Status code:", response.status_code)

# Chamando a função para extrair os dados das notícias do Brazil Journal
extrair_dados_brazil_journal()


                                              titulo  \
0  Vulcabrás tem ano histórico; Bartelle reitera ...   
1  Vulcabrás precifica follow-on a R$ 18,50 e lev...   
2  BREAKING: Vulcabrás prepara oferta de ações de...   
3  Vulcabrás dispara com dividendo gordo e ‘compr...   
4  Vulcabras bate consenso (de novo); “Esse é o n...   
5  Vulcabras: Itaú inicia com ‘compra’ e vê mais ...   
6  Vulcabrás bate o consenso, pensa em lojas próp...   

                                                 url  \
0  https://braziljournal.com/vulcabras-tem-ano-hi...   
1  https://braziljournal.com/vulcabras-precifica-...   
2  https://braziljournal.com/breaking-vulcabras-p...   
3  https://braziljournal.com/vulcabras-dispara-co...   
4  https://braziljournal.com/vulcabras-bate-conse...   
5  https://braziljournal.com/vulcabras-itau-inici...   
6  https://braziljournal.com/vulcabras-bate-o-con...   

                                                 img  \
0  https://braziljournal.com/wp-content/upload

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL para fazer a solicitação POST
url = "https://braziljournal.com/wp-admin/admin-ajax.php"

# Payload da solicitação POST
payload = {
    's': 'vulca',
    'action': 'searchwp_live_search',
    'swpengine': 'default',
    'swpquery': 'vulca',
    'origin_id': '322'
}

# Fazendo a solicitação POST
response = requests.post(url, data=payload)

# Verificando se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Obtendo o conteúdo HTML
    html = response.text

    # Parseando o HTML com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Encontrando todas as notícias
    articles = soup.find_all("article", class_="boxarticle")

    # Lista para armazenar os dados de todas as notícias
    all_data = []

    # Iterando sobre cada notícia para extrair título, link e imagem
    for article in articles:
        title_elem = article.find("h2", class_="boxarticle-infos-title")
        link_elem = article.find("a", class_="boxarticle-img")['href']
        img_elem = article.find("img")['src']

        # Definindo valores padrão para elementos ausentes
        title = title_elem.text.strip() if title_elem else "Título não disponível"
        link = link_elem if link_elem else "Link não disponível"
        img = img_elem if img_elem else "Imagem não disponível"

        # Adicionando os dados da notícia à lista de dados de todas as notícias
        all_data.append({
            "titulo": title,
            "url": link,
            "img": img,
            "fonte": "Brazil Journal"
        })

    # Criando o DataFrame com os dados de todas as notícias
    df_brazil_journal = pd.DataFrame(all_data)

    # Exibindo o DataFrame
    # print(df_brazil_journal)
else:
    print("Falha ao fazer a solicitação. Status code:", response.status_code)

df_brazil_journal


,titulo,url,img,fonte
0,Vulcabrás tem ano histórico; Bartelle reitera ...,https://braziljournal.com/vulcabras-tem-ano-hi...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
1,"Vulcabrás precifica follow-on a R$ 18,50 e lev...",https://braziljournal.com/vulcabras-precifica-...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
2,BREAKING: Vulcabrás prepara oferta de ações de...,https://braziljournal.com/breaking-vulcabras-p...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
3,Vulcabrás dispara com dividendo gordo e ‘compr...,https://braziljournal.com/vulcabras-dispara-co...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
4,Vulcabras bate consenso (de novo); “Esse é o n...,https://braziljournal.com/vulcabras-bate-conse...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
5,Vulcabras: Itaú inicia com ‘compra’ e vê mais ...,https://braziljournal.com/vulcabras-itau-inici...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
6,"Vulcabrás bate o consenso, pensa em lojas próp...",https://braziljournal.com/vulcabras-bate-o-con...,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


In [ ]:
def getNewsBrazilJournal(url, tag):

  # Inicializando o DataFrame com None
  df = None

  # Fazendo a solicitação GET
  response = requests.get(url)

  # Verificando se a solicitação foi bem-sucedida
  if response.status_code == 200:
      # Obtendo o conteúdo HTML
      html = response.text

      # Parseando o HTML com BeautifulSoup
      soup = BeautifulSoup(html, "html.parser")

      # Encontrando todas as notícias
      articles = soup.find_all("article", class_="boxarticle")

      # Lista para armazenar os dados de cada notícia
      data = []

      # Iterando sobre cada notícia para extrair título, link e data
      for article in articles:
          # print(article)
          title = article.find("h2", class_="boxarticle-infos-title").text.strip()
          link = article.find("h2", class_="boxarticle-infos-title").find("a")["href"]  # Extrai a URL da notícia
          img_elem = article.find("img")  # Encontra a tag img dentro da tag a
          img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir


          # Fazendo solicitação GET para o link da notícia
          response_article = requests.get(link)

          # Verificando se a solicitação para o link da notícia foi bem-sucedida
          if response_article.status_code == 200:
              # Obtendo o conteúdo HTML da notícia
              html_article = response_article.text

              # Parseando o HTML da notícia com BeautifulSoup
              soup_article = BeautifulSoup(html_article, "html.parser")
              # print(soup_article)

              # Encontrando a data de publicação da notícia, se existir
              post_time_elem = soup_article.find("div", class_="post-header-info")
              post_time = post_time_elem.text.strip() if post_time_elem else None


              # Adicionando os dados da notícia à lista de dados
              data.append({
                  "titulo": title,
                  "url": link,  # Agora a URL da notícia é usada aqui
                  "descricao": "",
                  "data_publicacao": post_time,
                  "img": img_url,
                  "fonte": "Brazil Journal"
              })
          else:
              print("Falha ao acessar o link da notícia:", link)

      # Criando o DataFrame com os dados das notícias
      df = pd.DataFrame(data)

  else:
      print("Falha ao acessar o site. Status code:", response.status_code)

  return df

# URL do site
urlBrazilJournal = [
        {"url": "https://braziljournal.com/tag/petroleo/", "tag": "petroleo"},
        {"url": "https://braziljournal.com/tag/setor-eletrico/", "tag": "setor-eletrico"},
        {"url": "https://braziljournal.com/tag/servicos-financeiros/", "tag": "servicos-financeiros"},
        {"url": "https://braziljournal.com/tag/varejo-e-consumo/", "tag": "varejo-e-consumo"},
        {"url": "https://braziljournal.com/tag/mineracao/", "tag": "mineracao"},
        {"url": "https://braziljournal.com/tag/siderurgia/", "tag": "siderurgia"},
        {"url": "https://braziljournal.com/tag/industria-automotiva/", "tag": "industria-automotiva"},
        {"url": "https://braziljournal.com/tag/agronegocio/", "tag": "agronegocio"},
        {"url": "https://braziljournal.com/tag/arte/", "tag": "arte"},
        {"url": "https://braziljournal.com/tag/beleza-e-cosmeticos/", "tag": "beleza-e-cosmeticos"},
        {"url": "https://braziljournal.com/tag/drogaria/", "tag": "drogaria"}
      ]

# Inicializando o DataFrame
df_bj = pd.DataFrame()

for url in urlBrazilJournal:
  df = getNewsBrazilJournal(url["url"], url["tag"])
  df_bj = pd.concat([df, df_bj])

# dropar linhas None de publicacao
df_bj = df_bj.dropna(subset=['data_publicacao'])

df_bj


Falha ao acessar o site. Status code: 404


,titulo,url,descricao,data_publicacao,img,fonte
0,Grupo Boticário cresce 30% e ganha share; B2B ...,https://braziljournal.com/grupo-boticario-cres...,,3 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
1,"No Mundo do Cabeleireiro, uma expansão adiada ...",https://braziljournal.com/no-mundo-do-cabeleir...,,6 de março de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
2,"Advent compra a Skala, a líder dos cremes para...",https://braziljournal.com/advent-compra-a-skal...,,28 de fevereiro de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
3,Boca Rosa — e agora livre. O voo solo da influ...,https://braziljournal.com/boca-rosa-e-agora-li...,,28 de dezembro de 2023,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
4,Natura&Co: The Body Shop já tem três interessados,https://braziljournal.com/natura-the-body-shop...,,20 de setembro de 2023,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
...,...,...,...,...,...,...
11,Lemann diz reavaliar governança e formação de ...,https://braziljournal.com/lemann-diz-reavaliar...,,6 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
12,A crise dos restaurantes estrelados,https://braziljournal.com/a-crise-dos-restaura...,,7 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
13,O que o iFood ‘roubou’ da Ambev; o plano do Ma...,https://braziljournal.com/o-que-o-ifood-roubou...,,6 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
14,OPINIÃO: “O Brasil é muito grande.” Mentira!,https://braziljournal.com/opiniao-o-brasil-e-m...,,7 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


In [ ]:
# URL do site
url = "https://braziljournal.com/"

# Fazendo a solicitação GET
response = requests.get(url)

# Verificando se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Obtendo o conteúdo HTML
    html = response.text

    # Parseando o HTML com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Encontrando todas as notícias
    articles = soup.find_all("article", class_="boxarticle")

    # Lista para armazenar os dados de cada notícia
    data = []

    # Iterando sobre cada notícia para extrair título, link e data
    for article in articles:
        title = article.find("h2", class_="boxarticle-infos-title").text.strip()
        link = article.find("a")["href"]
        img_elem = article.find("img")  # Encontra a tag img dentro da tag a
        img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir
        # tag = article.find("p", class_="boxarticle-infos-tag").text.strip()

        # Fazendo solicitação GET para o link da notícia
        response_article = requests.get(link)

        # Verificando se a solicitação para o link da notícia foi bem-sucedida
        if response_article.status_code == 200:
            # Obtendo o conteúdo HTML da notícia
            html_article = response_article.text

            # Parseando o HTML da notícia com BeautifulSoup
            soup_article = BeautifulSoup(html_article, "html.parser")

            # Encontrando a data de publicação da notícia, se existir
            post_time_elem = soup_article.find("div", class_="post-header-info")
            post_time = post_time_elem.text.strip() if post_time_elem else None

            # Adicionando os dados da notícia à lista de dados
            data.append({
                "titulo": title,
                "url": link,
                "descricao": "saiba mais a sobre "+title,
                "data_publicacao": post_time,
                "img": img_url,
                "fonte": "Brazil Journal"
            })
        else:
            print("Falha ao acessar o link da notícia:", link)

    # Criando o DataFrame com os dados das notícias
    df_bj_p = pd.DataFrame(data)

else:
    print("Falha ao acessar o site. Status code:", response.status_code)

df_bj_p = df_bj_p.dropna(subset=['data_publicacao'])
df_bj_p


,titulo,url,descricao,data_publicacao,img,fonte
42,Morgan Stanley é investigado por suspeita de l...,https://braziljournal.com/morgan-stanley-e-inv...,saiba mais a sobre Morgan Stanley é investigad...,11 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
47,Cadastra faz mais um M&A para crescer no ecomm...,https://braziljournal.com/cadastra-faz-mais-um...,saiba mais a sobre Cadastra faz mais um M&A pa...,10 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
52,Redoma Capital atrai Marcus Buaiz e Betina Roxo,https://braziljournal.com/redoma-capital-atrai...,saiba mais a sobre Redoma Capital atrai Marcus...,8 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
53,Nubank levanta US$ 150 milhões na Colômbia,https://braziljournal.com/nubank-levanta-us-15...,saiba mais a sobre Nubank levanta US$ 150 milh...,8 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


### Extrair Notícias do Valor Econômico

In [ ]:
# Lista para armazenar os dados de todas as páginas
all_data = []

# Loop sobre cada página até a página 100
for page_number in range(1, 101):
    # URL da página atual
    url = f"https://valor.globo.com/ultimas-noticias/index/feed/pagina-{page_number}"

    # Fazendo a solicitação GET
    response = requests.get(url)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Obtendo o conteúdo HTML
        html = response.text

        # Parseando o HTML com BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Encontrando todas as notícias
        articles = soup.find_all("div", class_="feed-post-body")

        # Iterando sobre cada notícia para extrair título, link, data e imagem
        for article in articles:
            title_elem = article.find("h2", class_="feed-post-link")
            link_elem = article.find("a")
            description_elem = article.find("p", class_="feed-post-body-resumo")
            img_elem = article.find("img")
            post_time_elem = article.find("span", class_="feed-post-datetime")

            # Definindo valores padrão para elementos ausentes
            title = title_elem.text.strip() if title_elem else "Título não disponível"
            link = link_elem["href"] if link_elem else "Link não disponível"
            description = description_elem.text.strip() if description_elem else "Descrição não disponível"
            img = img_elem["src"] if img_elem else "Imagem não disponível"
            post_time = post_time_elem.text.strip() if post_time_elem else "Data de publicação não disponível"

            # Adicionando os dados da notícia à lista de dados de todas as páginas
            all_data.append({
                "titulo": title,
                "url": link,
                "descricao": description,
                "data_publicacao": post_time,
                "img": img,
                "fonte": "Valor Econômico"
            })

        print(f"Página {page_number} processada.")
    else:
        print(f"Falha ao acessar a página {page_number}. Status code:", response.status_code)

# Criando o DataFrame com os dados de todas as páginas
df_valor = pd.DataFrame(all_data)

df_valor["data_publicacao"] = df_valor["data_publicacao"].apply(parse_relative_time)


Página 1 processada.
Página 2 processada.
Página 3 processada.
Página 4 processada.
Página 5 processada.
Página 6 processada.
Página 7 processada.
Página 8 processada.
Página 9 processada.
Página 10 processada.
Página 11 processada.
Página 12 processada.
Página 13 processada.
Página 14 processada.
Página 15 processada.
Página 16 processada.
Página 17 processada.
Página 18 processada.
Página 19 processada.
Página 20 processada.
Página 21 processada.
Página 22 processada.
Página 23 processada.
Página 24 processada.
Página 25 processada.
Página 26 processada.
Página 27 processada.
Página 28 processada.
Página 29 processada.
Página 30 processada.
Página 31 processada.
Página 32 processada.
Página 33 processada.
Página 34 processada.
Página 35 processada.
Página 36 processada.
Página 37 processada.
Página 38 processada.
Página 39 processada.
Página 40 processada.
Página 41 processada.
Página 42 processada.
Página 43 processada.
Página 44 processada.
Página 45 processada.
Página 46 processad

In [ ]:
# Lista para armazenar os dados de todas as páginas
all_data = []
lista_tag = ['empresas', 'brasil', 'politica', 'financas', 'mundo', 'agronegocios', 'legislacao', 'opiniao', 'carreira', 'eu-e', ]

for tag in lista_tag:
    print(f"Tag {tag}.")
    # Loop sobre cada página até a página 100
    for page_number in range(1, 101):
        # URL da página atual
        url = f"https://valor.globo.com/{tag}/index/feed/pagina-{page_number}"

        # Fazendo a solicitação GET
        response = requests.get(url)

        # Verificando se a solicitação foi bem-sucedida
        if response.status_code == 200:
            # Obtendo o conteúdo HTML
            html = response.text

            # Parseando o HTML com BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Encontrando todas as notícias
            articles = soup.find_all("div", class_="feed-post-body")

            # Iterando sobre cada notícia para extrair título, link, data e imagem
            for article in articles:
                title_elem = article.find("h2", class_="feed-post-link")
                link_elem = article.find("a")
                description_elem = article.find("p", class_="feed-post-body-resumo")
                img_elem = article.find("img")
                post_time_elem = article.find("span", class_="feed-post-datetime")

                # Definindo valores padrão para elementos ausentes
                title = title_elem.text.strip() if title_elem else "Título não disponível"
                link = link_elem["href"] if link_elem else "Link não disponível"
                description = description_elem.text.strip() if description_elem else "Descrição não disponível"
                img = img_elem["src"] if img_elem else "Imagem não disponível"
                post_time = post_time_elem.text.strip() if post_time_elem else "Data de publicação não disponível"

                # Adicionando os dados da notícia à lista de dados de todas as páginas
                all_data.append({
                    "titulo": title,
                    "url": link,
                    "descricao": description,
                    "data_publicacao": post_time,
                    "img": img,
                    "fonte": "Valor Econômico"
                })

            print(f"Página {page_number} processada.")
        else:
            print(f"Falha ao acessar a página {page_number}. Status code:", response.status_code)

# Criando o DataFrame com os dados de todas as páginas
df_valor = pd.DataFrame(all_data)

df_valor["data_publicacao"] = df_valor["data_publicacao"].apply(parse_relative_time)


Tag empresas.
Página 1 processada.
Página 2 processada.
Página 3 processada.
Página 4 processada.
Página 5 processada.
Página 6 processada.
Página 7 processada.
Página 8 processada.
Página 9 processada.
Página 10 processada.
Página 11 processada.
Página 12 processada.
Página 13 processada.
Página 14 processada.
Página 15 processada.
Página 16 processada.
Página 17 processada.
Página 18 processada.
Página 19 processada.
Página 20 processada.
Página 21 processada.
Página 22 processada.
Página 23 processada.
Página 24 processada.
Página 25 processada.
Página 26 processada.
Página 27 processada.
Página 28 processada.
Página 29 processada.
Página 30 processada.
Página 31 processada.
Página 32 processada.
Página 33 processada.
Página 34 processada.
Página 35 processada.
Página 36 processada.
Página 37 processada.
Página 38 processada.
Página 39 processada.
Página 40 processada.
Página 41 processada.
Página 42 processada.
Página 43 processada.
Página 44 processada.
Página 45 processada.
Págin

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Lista para armazenar os dados de todas as páginas
all_data = []
lista_empresa = ['Petroleo Brasileiro S.A. Petrobras']

for empresa in lista_empresa:

    for page_number in range(1, 6):
        # URL da página atual
        url = f"https://valor.globo.com/busca/?q={empresa}&page={page_number}&ajax=1"

        # Fazendo a solicitação GET
        response = requests.get(url)

        # Verificando se a solicitação foi bem-sucedida
        if response.status_code == 200:
            # Obtendo o conteúdo HTML
            html = response.text

            # Parseando o HTML com BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")

            # Encontrando todas as notícias
            articles = soup.find_all("li", class_="widget--card")

            # Iterando sobre cada notícia para extrair título, link, data e imagem
            for article in articles:
                title_elem = article.find("div", class_="widget--info__title")
                link_elem = article.find("a")
                description_elem = article.find("p", class_="widget--info__description")
                img_elem = article.find("img")
                post_time_elem = article.find("div", class_="widget--info__meta")

                # Definindo valores padrão para elementos ausentes
                title = title_elem.text.strip() if title_elem else None
                link = "https:"+link_elem['href'] if link_elem else None
                description = description_elem.text.strip() if description_elem else None
                img = "https:"+img_elem['src'] if img_elem else None
                post_time = post_time_elem.text.strip() if post_time_elem else None

                # Adicionando os dados da notícia à lista de dados de todas as páginas
                all_data.append({
                    "titulo": title,
                    "url": link,
                    "descricao": description,
                    "data_publicacao": post_time,
                    "img": img,
                    "fonte": "Valor"
                })

            print(f"Notícias da empresa {empresa} na página {page_number} processadas.")
        else:
            print(f"Falha ao acessar a página da empresa {empresa} na página {page_number}. Status code:", response.status_code)

# Criando o DataFrame com os dados de todas as páginas
df_valor = pd.DataFrame(all_data)

df_valor["data_publicacao"] = df_valor["data_publicacao"].apply(parse_relative_time)
df_valor

Notícias da empresa Petroleo Brasileiro S.A. Petrobras na página 1 processadas.
Notícias da empresa Petroleo Brasileiro S.A. Petrobras na página 2 processadas.
Notícias da empresa Petroleo Brasileiro S.A. Petrobras na página 3 processadas.
Notícias da empresa Petroleo Brasileiro S.A. Petrobras na página 4 processadas.
Notícias da empresa Petroleo Brasileiro S.A. Petrobras na página 5 processadas.


,titulo,url,descricao,data_publicacao,img,fonte
0,‘Máquina de dinheiro’ de bilionário apoiador d...,https://valor.globo.com/busca/click?q=Petroleo...,Uma recuperação no family office de John Pauls...,2024-04-16 18:20:04.598590,https://s2.glbimg.com/vdNgQEUt_7JbzxeTX1BLBhmQ...,Valor
1,Petróleo deve reagir com efeito limitado à esc...,https://valor.globo.com/busca/click?q=Petroleo...,"..., quando existem conflitos desse tipo o pre...",2024-04-15 23:20:04.598613,https://s2.glbimg.com/0BtMZio5CWRozPsU0DM62fVk...,Valor
2,"Usinas obsoletas, linhas de transmissão com ma...",https://valor.globo.com/busca/click?q=Petroleo...,... de 80% da produção de petróleo e derivados...,2024-04-15 23:20:04.598620,https://s2.glbimg.com/-M4LxM_YTQXwyNRcY0PQ4nEK...,Valor
3,SAS oferece cursos gratuitos em análise de dados,https://valor.globo.com/busca/click?q=Petroleo...,"O SAS, empresa de criação de software de IA e ...",2024-03-28 17:00,https://s2.glbimg.com/hkumoElV795p0B9dup3ZX1fX...,Valor
4,Combustíveis: nova fórmula requer ajustes,https://valor.globo.com/busca/click?q=Petroleo...,"... dos fretes”, observa.\nA Petrobras, maior ...",2024-03-28 08:05,https://s2.glbimg.com/QroJj1nxft6FuLeJBCz1FoiO...,Valor
...,...,...,...,...,...,...
70,"Ibovespa inicia pregão com foco em exterior, b...",https://valor.globo.com/busca/click?q=Petroleo...,"... brasileiro em Wall Street, subia 0,29% no ...",2023-07-31 15:39,None,Valor
71,"Agentes aguardam BCE, dados dos EUA e balanços...",https://valor.globo.com/busca/click?q=Petroleo...,"... noticiário doméstico, a elevação do rating...",2023-07-31 15:39,None,Valor
72,Movimento falimentar,https://valor.globo.com/busca/click?q=Petroleo...,"... fase\nPetrobras exporta 22,6% menos petról...",2023-08-01 13:02,None,Valor
73,Ibovespa deve iniciar pregão em alta com foco ...,https://valor.globo.com/busca/click?q=Petroleo...,"... brasileiro em Wall Street, subia 0,95% no ...",2023-08-07 20:14,None,Valor


In [26]:
df_valor.iloc[1].url

'http://valor.globo.com/busca/click?q=Petroleo+Brasileiro+S+A++Petrobras&p=2&r=1713319475073&u=https%3A%2F%2Fvalor.globo.com%2Fempresas%2Fnoticia%2F2024%2F04%2F14%2Fpetrleo-deve-reagir-com-efeito-limitado-escalada-de-tenso-no-oriente-mdio-dizem-especialistas.ghtml&syn=False&key=9f81126e4d6846cf4c1a5e8b7164d3a1'

### Extrair notícias Infomoney

In [ ]:
# Função para ler o XML da URL e retornar um DataFrame
def xml_to_dataframe(url):
    # Fazendo a requisição GET para obter o conteúdo da URL
    response = requests.get(url)

    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Analisando o XML
        root = ET.fromstring(response.content)

        # Inicializando listas para armazenar os dados
        titles = []
        links = []
        descriptions = []
        pub_dates = []
        images = []  # Nova lista para armazenar os URLs das imagens

        # Expressão regular para encontrar a tag <img> e seu conteúdo
        img_regex = re.compile(r'<img[^>]+src="([^"]+)"[^>]*>')

        # Iterando sobre os itens no XML
        for item in root.findall('.//item'):
            title = item.find('title').text
            link = item.find('link').text
            description = item.find('description').text.replace('<br />', '').replace("<p>", "").replace("</p>", "")
            pub_date = item.find('pubDate').text

            # Procurando por tags <img> na descrição e extraindo o URL da imagem
            img_match = img_regex.search(description)
            image_url = img_match.group(1) if img_match else None

            # Removendo a tag <img> da descrição
            description = img_regex.sub('', description)

            # Adicionando os dados às listas
            titles.append(title)
            links.append(link)
            descriptions.append(description)
            pub_dates.append(pub_date)
            images.append(image_url)

        # Criando um dicionário com os dados
        data = {
            'titulo': titles,
            'url': links,
            'descricao': descriptions,
            'data_publicacao': pub_dates,
            'img': images,
            "fonte": "InfoMoney"  # Alterando a fonte para "InfoMoney"
        }

        # Criando o DataFrame
        df = pd.DataFrame(data)
        return df
    else:
        print("Erro ao obter o XML. Código de status:", response.status_code)
        return None

# URL do XML do InfoMoney
url = "https://www.infomoney.com.br/feed/"

# Chamando a função para criar o DataFrame
df_infomoney = xml_to_dataframe(url)

# Exibindo o DataFrame
df_infomoney


,titulo,url,descricao,data_publicacao,img,fonte
0,Chile diz que morte de oposicionista venezuela...,https://www.infomoney.com.br/mundo/chile-diz-q...,"Em fevereiro, Ronald Ojeda, um ex-tenente vene...","Sat, 13 Apr 2024 15:00:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
1,Ozempic: investidor está otimista demais com r...,https://www.infomoney.com.br/onde-investir/oze...,A rápida aceitação de medicamentos do tipo cat...,"Sat, 13 Apr 2024 15:00:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
2,Irã apreende navio cargueiro associado a Israe...,https://www.infomoney.com.br/mundo/ira-apreend...,Comandos da Guarda Revolucionária Islâmica do ...,"Sat, 13 Apr 2024 14:51:12 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
3,Tribunal diz que ex-vice do Equador teve prisã...,https://www.infomoney.com.br/mundo/tribunal-di...,"Condenado duas vezes por corrupção, Jorge Glas...","Sat, 13 Apr 2024 14:11:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
4,Georgieva é aprovada para segundo mandato como...,https://www.infomoney.com.br/economia/georgiev...,A búlgara Kristalina Georgieva terá um novo ma...,"Sat, 13 Apr 2024 13:34:54 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
5,"Seguros, previdência e capitalização crescem 1...",https://www.infomoney.com.br/minhas-financas/s...,"Seguro fiança, que garante pagamento de alugué...","Sat, 13 Apr 2024 13:04:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
6,Índia vai às urnas com quase 1 bilhão de eleit...,https://www.infomoney.com.br/mundo/india-vai-a...,Como quase tudo o que envolve a população indi...,"Sat, 13 Apr 2024 13:00:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
7,Quanto o dinheiro rende na poupança hoje? Taxa...,https://www.infomoney.com.br/onde-investir/qua...,"A caderneta, que é isenta de IR, voltou a capt...","Sat, 13 Apr 2024 13:00:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
8,Justiça manda Enel SP ampliar atendimento huma...,https://www.infomoney.com.br/consumo/justica-m...,Exigências devem ser atendidas neste mês. Caso...,"Sat, 13 Apr 2024 12:04:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
9,Ataque a facadas mata seis pessoas em shopping...,https://www.infomoney.com.br/mundo/ataque-a-fa...,Um homem ainda não identificado esfaqueou as v...,"Sat, 13 Apr 2024 12:02:55 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney


### Concatenar as Fontes

In [ ]:
df = pd.concat([df_bj, df_bj_p, df_infomoney, df_valor])

def converter_data(valor):
    try:
        # Tenta converter para data diretamente
        return pd.to_datetime(valor)
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X dias atrás'
    if 'dias atrás' or 'dia atrás' in valor:
        dias_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(days=dias_atras)

    # Verifica se o valor está no formato 'X semanas atrás'
    if 'semanas atrás' or 'semana atrás' in valor:
        semanas_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(weeks=semanas_atras)

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d de %B de %Y')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d/%m/%Y %H:%M:%S')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%a, %d %b %Y %H:%M:%S %z')
    except ValueError:
        pass

    # Se não se encaixar em nenhum dos formatos conhecidos, retorna NaN
    return pd.NaT

# Aplicando a função de conversão à coluna 'data'
df['data_publicacao'] = df['data_publicacao'].apply(converter_data)

df

,titulo,url,descricao,data_publicacao,img,fonte
0,"Lunar Cruiser da Toyota, de transporte de astr...",https://valor.globo.com/empresas/noticia/2024/...,Veículo funcionará como um “carro de acampamen...,2024-04-13 16:14:16.280750,https://s2-valor.glbimg.com/_VpFoCDA0xxAuCwS4y...,Valor Econômico
1,"Após crise entre Musk e Moraes, responsável pe...",https://valor.globo.com/empresas/noticia/2024/...,"Até a noite desta sexta-feira, a ficha cadastr...",2024-04-13 13:24:16.280768,https://s2-valor.glbimg.com/g8OISKECRk2zfzh5Yd...,Valor Econômico
2,"Após lei, atendimentos por telemedicina cresce...",https://valor.globo.com/empresas/noticia/2024/...,Número passou de 11 milhões de atendimentos em...,2024-04-13 12:24:16.280774,https://s2-valor.glbimg.com/AkktmlB2ceLD7n-liE...,Valor Econômico
3,São Paulo tem quase um terço dos bares e resta...,https://valor.globo.com/empresas/noticia/2024/...,levantamento do Instituto FoodService Brasil m...,2024-04-13 11:24:16.280776,https://s2-valor.glbimg.com/lL4lPf7NEKQzKbJmXT...,Valor Econômico
4,Petrobras: Redução de prazo de pagamento a for...,https://valor.globo.com/empresas/noticia/2024/...,O presidente-executivo da associação avalia qu...,2024-04-13 02:24:16.280779,https://s2-valor.glbimg.com/5oYytvcY6B16Qz0NlR...,Valor Econômico
...,...,...,...,...,...,...
9949,Livros: Reflexões de uma ativista contra o ra...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Descrição não disponível,2023-07-18 16:24:16.310357,https://s2-valor.glbimg.com/bKrCFuR3_BpB7hrNLZ...,Valor Econômico
9950,Como um chef falido virou a mesa e se tornou o...,https://valor.globo.com/eu-e/noticia/2023/07/1...,"Há dez anos, o Boragó, de Rodolfo Guzmán, freq...",2023-07-18 16:24:16.310359,https://s2-valor.glbimg.com/N4kJgUGUd2KYGjqcok...,Valor Econômico
9951,"Guerra, covid e velhice compõem novo livro de ...",https://valor.globo.com/eu-e/noticia/2023/07/1...,"Em “Último olhar”, Miguel Sousa Tavares altern...",2023-07-18 16:24:16.310361,https://s2-valor.glbimg.com/CaxJwY6E-JMXW0iogl...,Valor Econômico
9952,Energia nuclear é menos destrutiva do que se i...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Jean-Marc Jancovici e o quadrinista Christophe...,2023-07-18 16:24:16.310364,https://s2-valor.glbimg.com/IGpzQOIWOCaOhcbmlI...,Valor Econômico


### Calculo de Similaridade - Verificar se as noticias estão repetindo

In [ ]:
# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['titulo'])

# Calculando a similaridade de cosseno entre os vetores TF-IDF
similaridade = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Definindo um limite de similaridade para considerar duas notícias como iguais
limite_similaridade = 0.8

# Lista para armazenar os índices das notícias únicas
indices_noticias_unicas = []

# Procurando por notícias semelhantes
for i in range(len(similaridade)):
    duplicada = False
    for j in range(i+1, len(similaridade)):
        if similaridade[i,j] > limite_similaridade:
            duplicada = True
            break
    if not duplicada:
        indices_noticias_unicas.append(i)

# Selecionando apenas as notícias únicas do dataframe original
df_noticias_unicas = df.iloc[indices_noticias_unicas]

# Agora 'df_noticias_unicas' contém apenas as notícias únicas
df = df_noticias_unicas

df

,titulo,url,descricao,data_publicacao,img,fonte
0,"Lunar Cruiser da Toyota, de transporte de astr...",https://valor.globo.com/empresas/noticia/2024/...,Veículo funcionará como um “carro de acampamen...,2024-04-13 16:14:16.280750,https://s2-valor.glbimg.com/_VpFoCDA0xxAuCwS4y...,Valor Econômico
1,"Após crise entre Musk e Moraes, responsável pe...",https://valor.globo.com/empresas/noticia/2024/...,"Até a noite desta sexta-feira, a ficha cadastr...",2024-04-13 13:24:16.280768,https://s2-valor.glbimg.com/g8OISKECRk2zfzh5Yd...,Valor Econômico
2,"Após lei, atendimentos por telemedicina cresce...",https://valor.globo.com/empresas/noticia/2024/...,Número passou de 11 milhões de atendimentos em...,2024-04-13 12:24:16.280774,https://s2-valor.glbimg.com/AkktmlB2ceLD7n-liE...,Valor Econômico
3,São Paulo tem quase um terço dos bares e resta...,https://valor.globo.com/empresas/noticia/2024/...,levantamento do Instituto FoodService Brasil m...,2024-04-13 11:24:16.280776,https://s2-valor.glbimg.com/lL4lPf7NEKQzKbJmXT...,Valor Econômico
4,Petrobras: Redução de prazo de pagamento a for...,https://valor.globo.com/empresas/noticia/2024/...,O presidente-executivo da associação avalia qu...,2024-04-13 02:24:16.280779,https://s2-valor.glbimg.com/5oYytvcY6B16Qz0NlR...,Valor Econômico
...,...,...,...,...,...,...
9949,Livros: Reflexões de uma ativista contra o ra...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Descrição não disponível,2023-07-18 16:24:16.310357,https://s2-valor.glbimg.com/bKrCFuR3_BpB7hrNLZ...,Valor Econômico
9950,Como um chef falido virou a mesa e se tornou o...,https://valor.globo.com/eu-e/noticia/2023/07/1...,"Há dez anos, o Boragó, de Rodolfo Guzmán, freq...",2023-07-18 16:24:16.310359,https://s2-valor.glbimg.com/N4kJgUGUd2KYGjqcok...,Valor Econômico
9951,"Guerra, covid e velhice compõem novo livro de ...",https://valor.globo.com/eu-e/noticia/2023/07/1...,"Em “Último olhar”, Miguel Sousa Tavares altern...",2023-07-18 16:24:16.310361,https://s2-valor.glbimg.com/CaxJwY6E-JMXW0iogl...,Valor Econômico
9952,Energia nuclear é menos destrutiva do que se i...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Jean-Marc Jancovici e o quadrinista Christophe...,2023-07-18 16:24:16.310364,https://s2-valor.glbimg.com/IGpzQOIWOCaOhcbmlI...,Valor Econômico


### Análise de Sentimento das notícias

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd

class SentimentClassifier:
    def __init__(self, model_name="lucas-leme/FinBERT-PT-BR"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)
        self.pred_mapper = {
            0: "positivo",
            1: "negativo",
            2: "neutro"
        }

    def classify_sentiments(self, df, text_column):
        results = []
        for text in df[text_column]:
            tokens = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = self.model(**tokens)
            probs = np.exp(outputs.logits.cpu().detach().numpy()) / np.exp(outputs.logits.cpu().detach().numpy()).sum(axis=1, keepdims=True)
            preds = dict(zip(self.pred_mapper.values(), probs[0]))
            results.append(preds)

        # Criar DataFrame com as colunas de probabilidades
        results_df = pd.DataFrame(results)

        # Adicionar as colunas de probabilidades ao DataFrame original usando join
        df_with_probs = df.join(results_df)

        return df_with_probs

# Exemplo de uso
# Supondo que 'df' é o seu DataFrame e 'text_column' é o nome da coluna que contém o texto das notícias
classifier = SentimentClassifier()
df = classifier.classify_sentiments(df, 'titulo')
df

,titulo,url,descricao,data_publicacao,img,fonte,positivo,negativo,neutro
0,"Lunar Cruiser da Toyota, de transporte de astr...",https://valor.globo.com/empresas/noticia/2024/...,Veículo funcionará como um “carro de acampamen...,2024-04-13 16:14:16.280750,https://s2-valor.glbimg.com/_VpFoCDA0xxAuCwS4y...,Valor Econômico,0.083320,0.066820,0.849860
1,"Após crise entre Musk e Moraes, responsável pe...",https://valor.globo.com/empresas/noticia/2024/...,"Até a noite desta sexta-feira, a ficha cadastr...",2024-04-13 13:24:16.280768,https://s2-valor.glbimg.com/g8OISKECRk2zfzh5Yd...,Valor Econômico,0.032679,0.867138,0.100183
2,"Após lei, atendimentos por telemedicina cresce...",https://valor.globo.com/empresas/noticia/2024/...,Número passou de 11 milhões de atendimentos em...,2024-04-13 12:24:16.280774,https://s2-valor.glbimg.com/AkktmlB2ceLD7n-liE...,Valor Econômico,0.780061,0.079365,0.140574
3,São Paulo tem quase um terço dos bares e resta...,https://valor.globo.com/empresas/noticia/2024/...,levantamento do Instituto FoodService Brasil m...,2024-04-13 11:24:16.280776,https://s2-valor.glbimg.com/lL4lPf7NEKQzKbJmXT...,Valor Econômico,0.585669,0.082625,0.331705
4,Petrobras: Redução de prazo de pagamento a for...,https://valor.globo.com/empresas/noticia/2024/...,O presidente-executivo da associação avalia qu...,2024-04-13 02:24:16.280779,https://s2-valor.glbimg.com/5oYytvcY6B16Qz0NlR...,Valor Econômico,0.043406,0.916601,0.039992
...,...,...,...,...,...,...,...,...,...
9949,Livros: Reflexões de uma ativista contra o ra...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Descrição não disponível,2023-07-18 16:24:16.310357,https://s2-valor.glbimg.com/bKrCFuR3_BpB7hrNLZ...,Valor Econômico,NaN,NaN,NaN
9950,Como um chef falido virou a mesa e se tornou o...,https://valor.globo.com/eu-e/noticia/2023/07/1...,"Há dez anos, o Boragó, de Rodolfo Guzmán, freq...",2023-07-18 16:24:16.310359,https://s2-valor.glbimg.com/N4kJgUGUd2KYGjqcok...,Valor Econômico,NaN,NaN,NaN
9951,"Guerra, covid e velhice compõem novo livro de ...",https://valor.globo.com/eu-e/noticia/2023/07/1...,"Em “Último olhar”, Miguel Sousa Tavares altern...",2023-07-18 16:24:16.310361,https://s2-valor.glbimg.com/CaxJwY6E-JMXW0iogl...,Valor Econômico,NaN,NaN,NaN
9952,Energia nuclear é menos destrutiva do que se i...,https://valor.globo.com/eu-e/noticia/2023/07/1...,Jean-Marc Jancovici e o quadrinista Christophe...,2023-07-18 16:24:16.310364,https://s2-valor.glbimg.com/IGpzQOIWOCaOhcbmlI...,Valor Econômico,NaN,NaN,NaN


In [ ]:
import pandas as pd
from openpyxl import Workbook

pd.api.types.is_datetime64_any_dtype(df['data_publicacao'])

if not pd.api.types.is_datetime64_any_dtype(df['data_publicacao']):
    df['data_publicacao'] = pd.to_datetime(df['data_publicacao'], utc=True)

df['data_publicacao'] = df['data_publicacao'].dt.tz_localize(None)

df.to_excel('notitcias.xlsx', index=False)

### Classificar a empresa

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Supondo que 'df_noticias' seja o seu dataframe contendo as notícias e os textos das notícias estejam na coluna 'titulo'
# Supondo que 'df_empresas' seja o seu dataframe contendo os nomes das empresas, os IDs das empresas e os nomes estejam nas colunas 'nome' e 'ID' respectivamente
df_noticias = df['titulo']
df_empresas = pd.DataFrame([
    {"nome": "Dexco", "ID": "123"},
    {"nome": "Petrobras", "ID": "124"},
    {"nome": "Tenda", "ID": "125"},
    {"nome": "Vale", "ID": "126"},
    {"nome": "Nubank", "ID": "127"}
])

# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix_noticias = vectorizer.fit_transform(df_noticias)

# Convertendo os nomes das empresas em vetores TF-IDF
nomes_empresas = [empresa['nome'] for empresa in df_empresas.to_dict(orient='records')]
tfidf_matrix_empresas = vectorizer.transform(nomes_empresas)

# Calculando a similaridade de cosseno entre os vetores TF-IDF das notícias e das empresas
similaridade = cosine_similarity(tfidf_matrix_noticias, tfidf_matrix_empresas)

# Definindo um limite de similaridade para considerar uma menção como uma citação de empresa
limite_similaridade = 0.20

# Lista para armazenar os índices das notícias que mencionam empresas
indices_noticias_com_citacoes = []

# Lista para armazenar as empresas mencionadas em cada notícia
empresas_mencionadas_por_noticia = []

# Procurando por notícias que mencionam empresas
for i in range(len(similaridade)):
    empresas_mencionadas = []
    for j in range(len(similaridade[i])):
        if similaridade[i,j] > limite_similaridade:
            empresas_mencionadas.append(df_empresas.iloc[j]['nome'])
    if empresas_mencionadas:
        indices_noticias_com_citacoes.append(i)
        empresas_mencionadas_por_noticia.append(empresas_mencionadas)

# Criando uma lista de dicionários para armazenar os resultados
resultado = []

# Preenchendo a lista com os dados
for empresas, indice in zip(empresas_mencionadas_por_noticia, indices_noticias_com_citacoes):
    for empresa in empresas:
        id_empresa = df_empresas[df_empresas['nome'] == empresa]['ID'].values[0]
        if re.match(r'\b[A-Z][a-z]*\b', empresa):  # Verifica se a primeira letra do nome da empresa é maiúscula
            resultado.append({'Empresa': empresa, 'ID': id_empresa, 'Notícia': df_noticias.iloc[indice]})

# Convertendo a lista de dicionários em um DataFrame
df_resultado = pd.DataFrame(resultado)

# Exibindo o DataFrame resultado
print(df_resultado)


     Empresa   ID                                            Notícia
0     Nubank  127  Nubank vai desativar app NuInvest e terá plata...
1  Petrobras  124  Dividendos da Petrobras (PETR4): acordo sobre ...
2       Vale  126  Ibovespa encerra em alta e se aproxima dos 129...
3     Nubank  127  Nubank obtém financiamento de US$ 150 milhões ...
4     Nubank  127  Nubank: Ações sobem 167% em um ano e banco dig...
5       Vale  126  Ibovespa avança mais de 1% com forte alta de V...
6     Nubank  127  Nubank lança conta global para clientes de alt...
7  Petrobras  124  Manhã no mercado: Petrobras e juros americanos...
